In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from datetime import datetime

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port

from mcs_shared import load_day, ALL_FLIGHT_DATES, load_snotel_locations, mcs_snotel_depth
from common import *

In [ ]:
%load_ext autoreload
%autoreload 2

## SNOTEL 

In [ ]:
snotel_sites = load_snotel_locations()
snotel_depth = mcs_snotel_depth(datetime(2020, 10, 1), datetime(2024, 7, 31))

## Flights

In [ ]:
mcs_dates = [pd.to_datetime(date) for date in ALL_FLIGHT_DATES]

In [ ]:
def get_flights(dates):
    flights = []
    means = []

    for flight in dates:
        mcs_als, isnobal, _diff, _flight_mask = load_day(flight)
        mean = float(mcs_als.snowdepth.mean().values)
        
        means.append(mean)
        flights.append(
            mcs_als.sel(
                x=snotel_sites.Mores_Creek_Summit.lon, 
                y=snotel_sites.Mores_Creek_Summit.lat,
                method='nearest',
            )       
        )

    return flights, means

In [ ]:
mcs_als, means = get_flights(ALL_FLIGHT_DATES)

In [ ]:
flight_values = [[date, float(flight.snowdepth)] for date, flight in zip(mcs_dates, mcs_als)]

## Plots

### Matplotlib

In [ ]:
als_values = [float(flight.snowdepth) for flight in mcs_als]

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(10,4))

snotel_depth.plot(
    label='Station', 
    ax=ax,
    color='seagreen', lw=1
)
ax.scatter(
    mcs_dates, als_values,
    s=20,
    color='black',
    label='ALS',
    zorder=10
)
ax.scatter(
    [mcs_dates[4], mcs_dates[5], mcs_dates[6], mcs_dates[10]], [als_values[4], als_values[5], als_values[6], als_values[10]],
    s=40,
    color='orange',
    zorder=10
)
ax.set_title('')

ax.set_ylabel('Snow Depth (m)')
ax.set_ylim(-0.1, 3)

ax.set_xlabel('')
ax.set_xlim(pd.to_datetime('2020-10-01'), pd.to_datetime('2024-07-31'))

ax.legend(ncols=3);

### Holoviews

In [ ]:
use_hvplot()

In [ ]:
hv.output(fig='auto', dpi=300)

HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
HV_PLOT_OPTS['xlabel'] = f'Water Year'

y_lim = (-0.05, 3.)
SNOTEL_LINE=dict(line_width=3)

snotel_depth.plot(label='Station', **SNOTEL_LINE, ylim=y_lim, **HV_PLOT_OPTS) * \
hv.Points(flight_values, label='Airborne').opts(color='black', size=10, tools=['hover'])